# Pre set up

In [3]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os
import cv2
import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 10,10
def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.show()

# Train ORCNN on amodal datasets

## Register my amodal datasets 

In [4]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
dataDir='datasets/coco'
annTrainFile='{}/annotations/COCO_amodal_train2014_with_classes_poly.json'.format(dataDir)
imgTrainFile = '{}/train2014'.format(dataDir)
register_coco_instances("amodal_coco_train", {},annTrainFile , imgTrainFile)
# Prepare test datasets 
annTestFile='{}/annotations/COCO_amodal_val2014_with_classes_poly.json'.format(dataDir)
imgTestFile = '{}/val2014'.format(dataDir)
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
register_coco_instances("amodal_coco_val", {}, annTestFile, imgTestFile)


## Sanity checks : overfitting small datasets 

In [ ]:
# Register small datasets for debugging 
from detectron2.data import DatasetCatalog,MetadataCatalog
dataset_dicts = DatasetCatalog.get("amodal_coco_train")
import random
imgs = random.sample(dataset_dicts,k=2)
DatasetCatalog.register("small_amodal_test", lambda : imgs)
metadata = {}
MetadataCatalog.get("small_amodal_test").set(
        image_root="datasets/coco/train2014", evaluator_type="coco", **metadata
    )


In [ ]:
# from pycocotools.coco import COCO
# coco_api = COCO(annTrainFile)
# smalldicts = DatasetCatalog.get("small_amodal_test")
# cat_ids = sorted(coco_api.getCatIds())
# cats = coco_api.loadCats(cat_ids)
# # The categories in a custom json file may not be sorted.
# tc = [c["name"] for c in sorted(cats, key=lambda x: x["id"])]
# print(tc)
# my_tc = []
# for i in range(len(smalldicts)):
#     for j in range(len(smalldicts[i]['annotations'])):
#         my_tc.append(smalldicts[i]['annotations'][j]['category_id'])
# my_tc = list( dict.fromkeys(my_tc) )
# my_tc = sorted(my_tc)
# print(my_tc)
# thing_classes = [tc[i] for i in my_tc]

In [ ]:
from pycocotools.coco import COCO
coco_api = COCO(annTrainFile)
cat_ids = sorted(coco_api.getCatIds())
cats = coco_api.loadCats(cat_ids)
# The categories in a custom json file may not be sorted.
thing_classes = [c["name"] for c in sorted(cats, key=lambda x: x["id"])]

In [ ]:
metadata = {}
MetadataCatalog.get("small_amodal_test").set(thing_classes = thing_classes, **metadata )
MetadataCatalog.get("small_amodal_test")

In [ ]:
smalldicts = DatasetCatalog.get("small_amodal_test")
for i in range(len(smalldicts)):
    im = cv2.imread(smalldicts[i]["file_name"])
    imshow(im[:, :, ::-1])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("small_amodal_test"), scale=0.5)
    vis = visualizer.draw_dataset_dict(smalldicts[i])
    imshow(vis.get_image()[:, :, ::-1])

### Training 


In [ ]:
import os
import numpy as np
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("small_amodal_test",)
cfg.DATASETS.TEST = ("small_amodal_test",)
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.STEPS = (1200,1400)
cfg.SOLVER.MAX_ITER = 1500 
cfg.VIS_PERIOD = 20
cfg.OUTPUT_DIR = "myAmodalCheckpoint"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Tensorboard

In [ ]:
!tensorboard --logdir=myAmodalCheckpoint --port=6006

### Evaluation

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import AmodalEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = AmodalEvaluator("small_amodal_test", cfg, False, output_dir="myAmodalEvaluation")
val_loader = build_detection_test_loader(cfg, "small_amodal_test")
# import pdb;pdb.set_trace()
inference_on_dataset(trainer.model, val_loader, evaluator)

### Visualization

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("small_amodal_test",)
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get("small_amodal_test")
for d in random.sample(dataset_dicts, 2):    
    im = cv2.imread(d["file_name"])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("small_amodal_test"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    imshow(vis.get_image()[:, :, ::-1])
#     import pdb;pdb.set_trace()
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("small_amodal_test"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    imshow(v.get_image()[:, :, ::-1])

## Training from a COCO-pretrained model as provided by Detectron2
1. Finetuning : For the case of COCOA amodal the final output layers that are class-specific had to be initialized randomly as the number of classes and their semantic meaning did not fit to the number of classes of COCO.

## Training 

In [6]:
import os
import numpy as np
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("amodal_coco_train",)
cfg.DATASETS.TEST = ("amodal_coco_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.SOLVER.BASE_LR = 0.0005  # pick a good LR
cfg.SOLVER.STEPS = (1300,1400)
cfg.SOLVER.MAX_ITER = 1500 
cfg.VIS_PERIOD = 1000
cfg.OUTPUT_DIR = "myAmodalCheckpoint"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[02/28 22:50:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[02/28 22:50:42 d2.data.datasets.coco]: Loaded 2276 images in COCO format from datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json
[02/28 22:50:42 d2.data.build]: Removed 0 images with no usable annotations. 2276 images left.
[02/28 22:50:42 d2.data.detection_utils]: TransformGens used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[02/28 22:50:42 d2.data.build]: Using training sampler TrainingSampler
[02/28 22:50:43 d2.engine.train_loop]: Starting training from iteration 0
[02/28 22:50:49 d2.utils.events]: eta: 0:06:41  iter: 19  total_loss: 1.066  loss_cls: 0.141  loss_box_reg: 0.125  loss_amodal_mask: 0.196  loss_visible_mask: 0.149  loss_invisible_mask: 0.477  loss_rpn_cls: 0.018  loss_rpn_loc: 0.006  time: 0.2754  data_time: 0.0134  lr: 0.000010  max_mem: 3382M
[02/28 22:50:55 d2.utils.events]: eta: 0:06:48  iter: 39  total_loss: 1.141  loss_cls: 0.173  loss_box_reg: 0.158  

[02/28 22:53:16 d2.utils.events]: eta: 0:04:45  iter: 519  total_loss: 1.104  loss_cls: 0.195  loss_box_reg: 0.195  loss_amodal_mask: 0.201  loss_visible_mask: 0.219  loss_invisible_mask: 0.166  loss_rpn_cls: 0.016  loss_rpn_loc: 0.012  time: 0.2916  data_time: 0.0036  lr: 0.000260  max_mem: 3721M
[02/28 22:53:22 d2.utils.events]: eta: 0:04:40  iter: 539  total_loss: 0.899  loss_cls: 0.179  loss_box_reg: 0.197  loss_amodal_mask: 0.176  loss_visible_mask: 0.130  loss_invisible_mask: 0.170  loss_rpn_cls: 0.006  loss_rpn_loc: 0.010  time: 0.2919  data_time: 0.0036  lr: 0.000270  max_mem: 3721M
[02/28 22:53:28 d2.utils.events]: eta: 0:04:34  iter: 559  total_loss: 0.866  loss_cls: 0.145  loss_box_reg: 0.201  loss_amodal_mask: 0.181  loss_visible_mask: 0.168  loss_invisible_mask: 0.110  loss_rpn_cls: 0.012  loss_rpn_loc: 0.011  time: 0.2920  data_time: 0.0040  lr: 0.000280  max_mem: 3721M
[02/28 22:53:34 d2.utils.events]: eta: 0:04:29  iter: 579  total_loss: 0.858  loss_cls: 0.173  loss_box

[02/28 22:55:59 d2.utils.events]: eta: 0:02:10  iter: 1059  total_loss: 0.869  loss_cls: 0.132  loss_box_reg: 0.164  loss_amodal_mask: 0.198  loss_visible_mask: 0.204  loss_invisible_mask: 0.116  loss_rpn_cls: 0.006  loss_rpn_loc: 0.011  time: 0.2958  data_time: 0.0036  lr: 0.000500  max_mem: 3721M
[02/28 22:56:04 d2.utils.events]: eta: 0:02:04  iter: 1079  total_loss: 0.909  loss_cls: 0.164  loss_box_reg: 0.195  loss_amodal_mask: 0.211  loss_visible_mask: 0.183  loss_invisible_mask: 0.131  loss_rpn_cls: 0.008  loss_rpn_loc: 0.010  time: 0.2955  data_time: 0.0038  lr: 0.000500  max_mem: 3721M
[02/28 22:56:11 d2.utils.events]: eta: 0:01:59  iter: 1099  total_loss: 0.924  loss_cls: 0.152  loss_box_reg: 0.202  loss_amodal_mask: 0.189  loss_visible_mask: 0.155  loss_invisible_mask: 0.116  loss_rpn_cls: 0.008  loss_rpn_loc: 0.013  time: 0.2957  data_time: 0.0036  lr: 0.000500  max_mem: 3721M
[02/28 22:56:17 d2.utils.events]: eta: 0:01:53  iter: 1119  total_loss: 0.830  loss_cls: 0.140  loss

WARNING [02/28 22:58:12 d2.engine.defaults]: No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.
[02/28 22:58:12 d2.utils.events]: eta: 0:00:00  iter: 1499  total_loss: 0.982  loss_cls: 0.133  loss_box_reg: 0.242  loss_amodal_mask: 0.198  loss_visible_mask: 0.206  loss_invisible_mask: 0.134  loss_rpn_cls: 0.003  loss_rpn_loc: 0.010  time: 0.2973  data_time: 0.0036  lr: 0.000005  max_mem: 3721M
[02/28 22:58:12 d2.engine.hooks]: Overall training speed: 1497 iterations in 0:07:25 (0.2975 s / it)
[02/28 22:58:12 d2.engine.hooks]: Total training time: 0:07:28 (0:00:03 on hooks)


In [8]:
!tensorboard --logdir=myAmodalCheckpoint --port=6006

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.1.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


## Evalution

In [9]:
cfg.DATASETS.TEST = ("amodal_coco_val",)
cfg.DATASETS.TRAIN = ("amodal_coco_train",)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import AmodalEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = AmodalEvaluator("amodal_coco_val", cfg, False, output_dir="myAmodalEvaluation")
val_loader = build_detection_test_loader(cfg, "amodal_coco_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

[02/28 22:59:41 d2.data.datasets.coco]: Loaded 1223 images in COCO format from datasets/coco/annotations/COCO_amodal_val2014_with_classes_poly.json
[02/28 22:59:41 d2.evaluation.evaluator]: Start inference on 1223 images
[02/28 22:59:42 d2.evaluation.evaluator]: Inference done 11/1223. 0.0542 s / img. ETA=0:01:11
[02/28 22:59:47 d2.evaluation.evaluator]: Inference done 97/1223. 0.0546 s / img. ETA=0:01:05
[02/28 22:59:52 d2.evaluation.evaluator]: Inference done 184/1223. 0.0545 s / img. ETA=0:01:00
[02/28 22:59:57 d2.evaluation.evaluator]: Inference done 269/1223. 0.0547 s / img. ETA=0:00:55
[02/28 23:00:02 d2.evaluation.evaluator]: Inference done 353/1223. 0.0550 s / img. ETA=0:00:51
[02/28 23:00:07 d2.evaluation.evaluator]: Inference done 439/1223. 0.0550 s / img. ETA=0:00:46
[02/28 23:00:12 d2.evaluation.evaluator]: Inference done 524/1223. 0.0551 s / img. ETA=0:00:41
[02/28 23:00:17 d2.evaluation.evaluator]: Inference done 608/1223. 0.0551 s / img. ETA=0:00:36
[02/28 23:00:22 d2.ev

Loading and preparing results...
DONE (t=0.06s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *visible*
DONE (t=2.34s).
Accumulating evaluation results...
DONE (t=0.77s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.312
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.405
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.360
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.277
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.409
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.287
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.354
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.354
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets

OrderedDict([('bbox',
              {'AP': 30.68075979372943,
               'AP50': 39.36921086052871,
               'AP75': 35.005607718478956,
               'APs': 12.810229683560658,
               'APm': 28.411168559161876,
               'APl': 38.65940296878257,
               'AP-person': 51.22700060879134,
               'AP-bicycle': 12.871287128712867,
               'AP-car': 22.640482067001976,
               'AP-motorcycle': 38.15055265061089,
               'AP-airplane': 64.12932226362629,
               'AP-bus': 54.867605009392015,
               'AP-train': 53.04591977707602,
               'AP-truck': 26.322510192195693,
               'AP-boat': 16.155073460723536,
               'AP-traffic light': 2.376237623762376,
               'AP-fire hydrant': 69.28799912958328,
               'AP-stop sign': 10.862800565770863,
               'AP-parking meter': 0.0,
               'AP-bench': 41.045162410977944,
               'AP-bird': 27.792197076850545,
            

## Visualization

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("amodal_coco_val",)
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get("amodal_coco_val")
for d in random.sample(dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("amodal_coco_val"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    imshow(vis.get_image()[:, :, ::-1])
#     import pdb;pdb.set_trace()
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("amodal_coco_val"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    imshow(v.get_image()[:, :, ::-1])